In [1]:
# Query Wikidata & download results as tsv  
# This script also generates the information for Table 1 in the paper
#
# 23 June  - 12 July 2024
# marieke.van.erp@dh.huc.knaw.nl

In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

In [3]:
sparql = SPARQLWrapper("https://query.wikidata.org/bigdata/namespace/wdq/sparql")
sparql.setReturnFormat(JSON)


In [13]:
def datastats(dataframe):
    dataframe['occurrence_companies'] = dataframe.groupby('company.value')['company.value'].transform('size')
    number_of_entities = dataframe['company.value'].nunique()
    number_of_statements = len(dataframe.index),
    minimum = dataframe['occurrence_companies'].min()
    maximum = dataframe['occurrence_companies'].max()
    mean = dataframe['occurrence_companies'].mean()
    median = dataframe['occurrence_companies'].median()
    stdev = dataframe['occurrence_companies'].std()
    stats_result = str(number_of_entities) + " & " + str(number_of_statements) + " & " + str(minimum) + " & " + str(maximum) + " & " + str(mean) +" & " + str(median) + " & " + str(stdev) + "\\"
    return(stats_result)

In [9]:
# This query selects for each company the number of employees at a given point in time 
sparql.setQuery("""
SELECT ?company ?companyLabel ?numEmployees ?point
WHERE
{
  VALUES ?p { wd:Q6881511 wd:Q4830453 } .  
  ?company wdt:P31 ?p .
  ?company p:P1128 ?employees .         
  ?employees ps:P1128 ?numEmployees .
  ?employees pq:P585 ?point .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" . }
} 
   """
)

try:
    results = sparql.queryAndConvert()
#    for result in results["results"]["bindings"]:
#        print(result)
except Exception as e:
    print(e)

In [14]:
# Convert results to a dataframe and save as tsv for further analysis
results_df = pd.json_normalize(results['results']['bindings'])
results_df = results_df.loc[:,~results_df.columns.str.endswith('type')]
results_df.to_csv("company_employees.tsv", sep = "\t")
print(datastats(results_df))

5764 & (13090,) & 1 & 58 & 7.283422459893048 & 2.0 & 11.075389847313827\


In [15]:
# This query selects for each company the ceo and their start date 
# wd:Q6881511 enterprise (subclass of:)
# wd:Q4830453 business 
sparql.setQuery("""SELECT ?company ?companyLabel ?ceoname ?ceonameLabel ?starttime
WHERE
{
  VALUES ?p { wd:Q6881511 wd:Q4830453 } .  
  ?company wdt:P31 ?p .
  ?company p:P169 ?ceo .         
  ?ceo ps:P169 ?ceoname .
  ?ceo pq:P580 ?starttime .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" . }
}
"""
)

try:
    results = sparql.queryAndConvert()
except Exception as e:
    print(e)               

In [16]:
# Convert results to a dataframe and save as tsv for further analysis
results_df = pd.json_normalize(results['results']['bindings'])
results_df = results_df.loc[:,~results_df.columns.str.endswith('type')]
results_df.to_csv("company_ceo_starttime.tsv", sep = "\t")
print(datastats(results_df))

1252 & (2829,) & 1 & 26 & 4.910922587486745 & 2.0 & 5.165286170258831\


In [19]:
# This query selects for each company their subsidiaries and their start date 
# wd:Q6881511 enterprise (subclass of:)
# wd:Q4830453 business 
sparql.setQuery("""SELECT ?company ?companyLabel ?subsidiaryname ?subsidiarynameLabel ?starttime
WHERE
{
  VALUES ?p { wd:Q6881511 wd:Q4830453 } .  
  ?company wdt:P31 ?p .
  ?company p:P355 ?subsidiary .         
  ?subsidiary ps:P355 ?subsidiaryname .
  ?subsidiary pq:P580|pq:P585 ?starttime .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" . }
} 
"""
)


try:
    results = sparql.queryAndConvert()
except Exception as e:
    print(e)  

In [20]:
# Convert results to a dataframe and save as tsv for further analysis
results_df = pd.json_normalize(results['results']['bindings']) 
results_df = results_df.loc[:,~results_df.columns.str.endswith('type')]
results_df.to_csv("company_subsidiary_starttime.tsv", sep = "\t")
print(datastats(results_df))

708 & (1548,) & 1 & 28 & 4.79328165374677 & 2.0 & 5.296987845809616\


In [22]:
# This query selects for each company the companies they own and their start date/given point in time
sparql.setQuery("""SELECT ?company ?companyLabel ?othercompanyname ?othercompanynameLabel ?starttime
WHERE
{
  VALUES ?p { wd:Q6881511 wd:Q4830453 } .  
  ?company wdt:P31 ?p .
  ?company p:P1830 ?othercompany .         
  ?othercompany ps:P1830 ?othercompanyname .
  ?othercompany pq:P580|pq:P585  ?starttime .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" . }
} 
"""
)                
                
try:
    results = sparql.queryAndConvert()
except Exception as e:
    print(e) 

In [23]:
# Convert results to a dataframe and save as tsv for further analysis
results_df = pd.json_normalize(results['results']['bindings']) 
results_df = results_df.loc[:,~results_df.columns.str.endswith('type')]
results_df.to_csv("company_owns_starttime.tsv", sep = "\t")
print(datastats(results_df))

506 & (846,) & 1 & 18 & 3.2671394799054374 & 2.0 & 3.698012656354469\


In [24]:
# This query selects for each company the subsidiaries they own and their start date/given point in time,
# as well as the industry/industries of the original company and of the subsidiaries 

sparql.setQuery("""SELECT ?company ?companyLabel ?subsidiaryId ?subsidiaryIdLabel ?starttime ?industry ?industryLabel ?industry2 ?industry2Label
WHERE
{
  VALUES ?p { wd:Q6881511 wd:Q4830453 } .  
  ?company wdt:P31 ?p .
  ?company wdt:P452 ?industry .
  ?company p:P355 ?subsidiaryStmt .         
  ?subsidiaryStmt ps:P355 ?subsidiaryId .
  ?subsidiaryStmt pq:P580|pq:P585 ?starttime .
optional { ?subsidiaryId wdt:P452 ?industry2 . } 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" . }
} 
"""
)                
                
try:
    results = sparql.queryAndConvert()
except Exception as e:
    print(e) 

In [25]:
# Convert results to a dataframe and save as tsv for further analysis
results_df = pd.json_normalize(results['results']['bindings']) 
results_df = results_df.loc[:,~results_df.columns.str.endswith('type')]
results_df.to_csv("company_subsidiary_industry.tsv", sep = "\t")
print(datastats(results_df))

518 & (3017,) & 1 & 96 & 27.142525687769307 & 12.0 & 28.199099786423588\


In [26]:
# This query gathers the company's official name(s) and times during which that name/those names were valid 

sparql.setQuery("""SELECT ?company ?companyLabel ?name ?starttime ?endtime
WHERE
{ 
  VALUES ?p { wd:Q6881511 wd:Q4830453 } .  
  ?company wdt:P31 ?p .
  ?company p:P1448 ?nameId. 
  ?nameId ps:P1448 ?name .
  ?nameId pq:P580|pq:P585 ?starttime .
  OPTIONAL { ?nameId pq:P582 ?endtime . }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" . }
}
"""
)                
                
try:
    results = sparql.queryAndConvert()
except Exception as e:
    print(e) 

In [27]:
# Convert results to a dataframe and save as tsv for further analysis
results_df = pd.json_normalize(results['results']['bindings']) 
results_df = results_df.loc[:,~results_df.columns.str.endswith('type')]
results_df.to_csv("company_names_dates.tsv", sep = "\t")
print(datastats(results_df))

1643 & (4790,) & 1 & 18 & 4.389144050104385 & 4.0 & 2.928201720235181\


In [30]:
# This query gathers the company's mottos/motto texts and times during which that motto/those mottos were valid 
# motto text : P1451 
# motto : P1546 

sparql.setQuery("""SELECT ?company ?companyLabel ?mottoId ?mottoLabel ?starttime ?endtime
WHERE
{ 
  VALUES ?p { wd:Q6881511 wd:Q4830453 } .  
  ?company wdt:P31 ?p .
  ?company p:P1546|p:P1451 ?mottoId. 
  ?mottoId ps:P1546|ps:P1451 ?motto .
  ?mottoId pq:P580|pq:P585 ?starttime . 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" . }
}
"""
)                
                
try:
    results = sparql.queryAndConvert()
except Exception as e:
    print(e) 

In [31]:
# Convert results to a dataframe and save as tsv for further analysis
results_df = pd.json_normalize(results['results']['bindings']) 
results_df = results_df.loc[:,~results_df.columns.str.endswith('type')]
results_df.to_csv("company_mottos_dates.tsv", sep = "\t")
print(datastats(results_df))

29 & (56,) & 1 & 4 & 2.5 & 2.0 & 1.1441551070947107\


In [32]:
# This query gathers the company's acronyms and times during which that motto/those mottos were valid 
# short name : P1813

sparql.setQuery("""SELECT ?company ?companyLabel ?acronymId ?acronym ?starttime ?endtime
WHERE
{ 
  VALUES ?p { wd:Q6881511 wd:Q4830453 } .  
  ?company wdt:P31 ?p .
  ?company p:P1813 ?acronymId . 
  ?acronymId ps:P1813 ?acronym .
  OPTIONAL { ?acronymId pq:P580|pq:P585 ?starttime . }
  OPTIONAL { ?acronymId pq:P582 ?endtime . }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" . }
}
"""
)                
                
try:
    results = sparql.queryAndConvert()
except Exception as e:
    print(e) 

In [33]:
# Convert results to a dataframe and save as tsv for further analysis
results_df = pd.json_normalize(results['results']['bindings']) 
#results_df = results_df.loc[:,~results_df.columns.str.endswith('type')]
results_df.to_csv("company_acronyms_dates_with_types.tsv", sep = "\t")
print(datastats(results_df))

1250 & (1728,) & 1 & 9 & 1.8310185185185186 & 1.0 & 1.3264616562208242\
